In [18]:
import gensim
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import os
import re

In [8]:
data_folder = "data/ReligiousTexts/"
files = os.listdir(data_folder)

In [5]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [28]:
texts = []
for file in files:
    with open(data_folder+file,encoding="ISO-8859-1") as f:
        texts.append(   re.split('[, \n]',f.read())  )

In [30]:
common_dictionary = Dictionary(texts)

In [31]:
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [45]:
lda = gensim.models.LdaModel(common_corpus,id2word=common_dictionary ,num_topics=150)

In [46]:
lda.get_document_topics(common_dictionary.doc2bow(texts[0]))

[(33, 0.30530244), (75, 0.67804253), (142, 0.016398354)]

In [51]:
for t in lda.get_topic_terms(57, topn=100):
    print(lda.id2word[t[0]],t[1],t[0])

 0.09190485 0
the 0.054985255 815
of 0.037572064 761
and 0.031291462 647
is 0.011784585 733
to 0.011747599 823
that 0.011560172 814
in 0.009749925 724
a 0.0091828415 636
be 0.006767639 654
I 0.006432521 345
with 0.0061803535 843
as 0.006123986 650
he 0.005875078 12675
for 0.0050965482 702
his 0.0049729915 718
not 0.0047914954 15258
which 0.0047436818 840
are 0.004721928 649
they 0.0045778984 17622
And 0.004509206 5090
by 0.0041300305 665
from 0.004126268 706
it 0.0039078356 734
; 0.0038449296 233
them 0.0038106556 17602
have 0.0038074453 715
: 0.0037534314 227
was 0.003655175 837
all 0.0036539645 9882
on 0.0035051007 763
this 0.00350335 818
him 0.0031659685 12800
Lord 0.002940864 6935
. 0.0027647 56
their 0.0025885142 17599
The 0.0025861962 537
shall 0.0025223768 16923
were 0.0025078577 18473
we 0.002468393 18447
but 0.0024129038 664
one 0.0023919037 764
God 0.0023272948 6148
unto 0.0023167033 18185
who 0.0022345402 18489
or 0.002106968 765
will 0.0020667876 18504
there 0.0020217823 17